In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/airy-post-8/airy_post_car_rest.ckpt
/kaggle/input/asba-nlim/airy_test_nlim.csv
/kaggle/input/asba-nlim/airy_test_nlim2.csv
/kaggle/input/asba-nlim/rest_test_nlim.csv
/kaggle/input/asba-nlim/car_train_nlim.csv
/kaggle/input/asba-nlim/rest_train_nlim.csv
/kaggle/input/asba-nlim/airy_train_nlim.csv
/kaggle/input/asba-nlim/car_test_nlim.csv


In [2]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2023-02-10 07:55:31.633989: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 07:55:31.705333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-10 07:55:31.820135: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-10 07:55:31.820971: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [3]:
!pip install sentencepiece
!pip install pytorch-transformers
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 1.2 MB/s eta 0:00:00


In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


from pytorch_transformers import XLNetModel
from transformers import XLNetForSequenceClassification, XLNetTokenizer
from pytorch_transformers import AdamW
from random import random
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

%matplotlib inline

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [6]:
def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens):
    """Truncates a pair of sequences to a maximum sequence length. Lifted from Google's BERT repo."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_num_tokens:
            break

        trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
        assert len(trunc_tokens) >= 1

        # We want to sometimes truncate from the front and sometimes from the
        # back to add more randomness and avoid biases.
        if random() < 0.5:
            del trunc_tokens[0]
        else:
            trunc_tokens.pop()

In [7]:
def get_tokenized_texts_from(tokenizer, sentences_a, sentences_b, max_len):
    tokenized_texts_a = [tokenizer.tokenize(sent) for sent in sentences_a]
    tokenized_texts_b = [tokenizer.tokenize(sent) for sent in sentences_b]
    tokenized_texts = []
    for i in range (len(tokenized_texts_a)):
        truncate_seq_pair(tokenized_texts_a[i], tokenized_texts_b[i], max_len-3)
        tokens = tokenized_texts_a[i] + ["<sep>"] + tokenized_texts_b[i] + ["<sep>"] + ["<cls>"]
        tokenized_texts.append(tokens)
    return tokenized_texts

In [8]:
df_test1 = pd.read_csv("/kaggle/input/asba-nlim/airy_test_nlim.csv")
df_test2 = pd.read_csv("/kaggle/input/asba-nlim/airy_test_nlim2.csv")

In [9]:
MAX_LEN = 128

In [10]:
sentences_a_test1 = df_test1.text_a.values
sentences_b_test1 = df_test1.text_b.values
labels_test1 = df_test1.label.values

In [11]:
sentences_a_test2 = df_test2.text_a.values
sentences_b_test2 = df_test2.text_b.values
labels_test2 = df_test2.label.values

In [12]:
tokenizer = XLNetTokenizer.from_pretrained('malay-huggingface/xlnet-base-bahasa-cased', do_lower_case=True)

tokenized_texts_test1 = get_tokenized_texts_from(tokenizer, sentences_a_test1, sentences_b_test1, MAX_LEN)
tokenized_texts_test2 = get_tokenized_texts_from(tokenizer, sentences_a_test2, sentences_b_test2, MAX_LEN)


Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/290 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

In [13]:
input_ids_test1 = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_test1]
input_ids_test2 = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_test2]

In [14]:
input_ids_test1 = pad_sequences(input_ids_test1, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids_test2 = pad_sequences(input_ids_test2, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [15]:
tokenized_texts_test1 =pad_sequences(tokenized_texts_test1, maxlen=MAX_LEN, dtype=object, truncating="post", padding="post")

df_test1['tokens'] = tokenized_texts_test1.tolist()

In [16]:
tokenized_texts_test2 =pad_sequences(tokenized_texts_test2, maxlen=MAX_LEN, dtype=object, truncating="post", padding="post")

df_test2['tokens'] = tokenized_texts_test2.tolist()

In [17]:
attention_masks_test1 = []
for seq in input_ids_test1:
  seq_mask_test = [float(i>0) for i in seq]
  attention_masks_test1.append(seq_mask_test)
    
attention_masks_test2 = []
for seq in input_ids_test2:
  seq_mask_test = [float(i>0) for i in seq]
  attention_masks_test2.append(seq_mask_test)

In [18]:
test_inputs1 = torch.tensor(input_ids_test1)
test_labels1 = torch.tensor(labels_test1)
test_masks1 = torch.tensor(attention_masks_test1)

test_inputs2 = torch.tensor(input_ids_test2)
test_labels2 = torch.tensor(labels_test2)
test_masks2 = torch.tensor(attention_masks_test2)

In [19]:
batch_size = 32
test_data1 = TensorDataset(test_inputs1, test_masks1, test_labels1)
test_sampler1 = SequentialSampler(test_data1)
test_dataloader1 = DataLoader(test_data1, sampler=test_sampler1, batch_size=batch_size)

test_data2 = TensorDataset(test_inputs2, test_masks2, test_labels2)
test_sampler2 = SequentialSampler(test_data2)
test_dataloader2 = DataLoader(test_data2, sampler=test_sampler2, batch_size=batch_size)

In [20]:
model = XLNetForSequenceClassification.from_pretrained("malay-huggingface/xlnet-base-bahasa-cased", num_labels=2)
model.load_state_dict(torch.load("/kaggle/input/airy-post-8/airy_post_car_rest.ckpt"))
model.to(device)

Downloading:   0%|          | 0.00/686 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at malay-huggingface/xlnet-base-bahasa-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at malay-huggingface/xlnet-base-bahasa-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [21]:
predictions1 = []
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(test_dataloader1):
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      # print (outputs)
      prediction = torch.argmax(outputs[0],dim=1)
      predictions1 = predictions1 + prediction.tolist()
      total += b_labels.size(0)
      correct+=(prediction==b_labels).sum().item()

In [22]:
print('Test Accuracy of the finetuned model on val data is: {} %'.format(100 * correct / total))
print('F1 Macro of the finetuned model on val data is: {} %'.format(100 * f1_score(labels_test1, predictions1, average='macro')))

Test Accuracy of the finetuned model on val data is: 97.99196787148594 %
F1 Macro of the finetuned model on val data is: 97.01401255553756 %


In [23]:
predictions2 = []
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(test_dataloader2):
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      # print (outputs)
      prediction = torch.argmax(outputs[0],dim=1)
      predictions2 = predictions2 + prediction.tolist()
      total += b_labels.size(0)
      correct+=(prediction==b_labels).sum().item()

In [24]:
print('Test Accuracy of the finetuned model on val data is: {} %'.format(100 * correct / total))
print('F1 Macro of the finetuned model on val data is: {} %'.format(100 * f1_score(labels_test2, predictions2, average='macro')))

Test Accuracy of the finetuned model on val data is: 93.91249207355739 %
F1 Macro of the finetuned model on val data is: 93.85616331619137 %


In [25]:
df_test1["pred"] = predictions1
df_test2["pred"] = predictions2

In [26]:
df_test1.to_csv("prediction.csv", index=False)
df_test2.to_csv("prediction2.csv", index=False)